In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler

# Veriyi okuma
file_path = "/content/drive/MyDrive/Colab Notebooks/visea/train2.xlsx"
data = pd.read_excel(file_path)

data['DateTime'] = pd.to_datetime(data['DateTime'])
data.set_index('DateTime', inplace=True)

# Veriyi ölçeklendirme
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[['Distributed Energy']])

# Zaman serisi verisini LSTM için uygun hale getirme
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length])
        labels.append(data[i+seq_length])
    return np.array(sequences), np.array(labels)


# Son 24 saatlik veriyi kullanarak tahmin yap
seq_length = 24
train_size = int(len(data_scaled) * 0.9)
train_data = data_scaled[:train_size]
val_data = data_scaled[train_size - seq_length:]  # Geçmiş bilgiyi de içermeli

X_train, y_train = create_sequences(train_data, seq_length)
X_val, y_val = create_sequences(val_data, seq_length)

# Model Tanımlama
model = Sequential([
    Input(shape=(seq_length, 1)),
    Bidirectional(LSTM(100, return_sequences=True)),
    BatchNormalization(),
    Dropout(0.3),
    Bidirectional(LSTM(100, return_sequences=True)),
    BatchNormalization(),
    Dropout(0.3),
    LSTM(50, return_sequences=False),
    BatchNormalization(),
    Dense(50, activation='relu'),
    Dense(25, activation='relu'),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Early Stopping ve Learning Rate Reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

epochs = 100
batch_size = 32
model.fit(X_train, y_train,
          validation_data=(X_val, y_val),
          epochs=epochs,
          batch_size=batch_size,
          callbacks=[early_stopping, reduce_lr])





Epoch 1/100
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 34s 18ms/step - loss: 0.0357 - val_loss: 0.0080 - learning_rate: 0.0010
Epoch 2/100
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 36s 16ms/step - loss: 0.0034 - val_loss: 0.0018 - learning_rate: 0.0010
Epoch 3/100
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - loss: 0.0024 - val_loss: 0.0034 - learning_rate: 0.0010
Epoch 4/100
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - loss: 0.0017 - val_loss: 0.0013 - learning_rate: 0.0010
Epoch 5/100
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - loss: 0.0015 - val_loss: 0.0012 - learning_rate: 0.0010
Epoch 6/100
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - loss: 0.0011 - val_loss: 9.7883e-04 - learning_rate: 0.0010
Epoch 7/100
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - loss: 0.0010 - val_loss: 7.8705e-04 - learning_rate: 0.0010
Epoch 8/100
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - loss: 8.4684e-04 - val_loss: 9.4774e-04 - learning_rate: 0.0010
Epoch 9/100
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - loss:

In [ ]:
# Son 24 saatlik veriyi alarak 48 saat tahmin yapma
forecast_input = data_scaled[-seq_length:].reshape(1, seq_length, 1)
forecast = []

for _ in range(48):  # 48 saatlik tahmin
    next_hour = model.predict(forecast_input, verbose=0)
    forecast.append(next_hour[0, 0])
    forecast_input = np.append(forecast_input[:, 1:, :], next_hour.reshape(1, 1, 1), axis=1)

# Tahminleri ters ölçeklendirme
forecast_rescaled = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))

# Sonuçları kaydetme
forecast_dates = pd.date_range(start=data.index[-1], periods=49, freq='h')[1:]  # Tahmin tarihleri
forecast_df = pd.DataFrame({'DateTime': forecast_dates, 'Forecasted Value': forecast_rescaled.flatten()})
output_file = "/content/drive/MyDrive/Colab Notebooks/visea/forecast_lstm_results_model2_seq24.xlsx"
forecast_df.to_excel(output_file, index=False)

# Tahmin sonuçlarını değerlendirme
print(f"LSTM Forecast saved to {output_file}")

# Modeli kaydetme
model.save("/content/drive/MyDrive/Colab Notebooks/visea/lstm_forecast_model2_seq24.keras")
print(X_train.shape, " X_train")
print(X_val.shape, "X_val")

from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score

# Tahminler yapılır
y_val_pred = model.predict(X_val)
# Tahminlerinizi ve gerçek değerlerinizi scaler ile tersine çevirebilirsiniz
y_val_pred_inverse = scaler.inverse_transform(y_val_pred.reshape(-1, 1))
y_val_inverse = scaler.inverse_transform(y_val.reshape(-1, 1))

# Hata metriklerinizi tekrar hesaplayın
r2 = r2_score(y_val_inverse, y_val_pred_inverse)
print(f"Validation R²: {r2:.4f}")

rmse = np.sqrt(mean_squared_error(y_val_inverse, y_val_pred_inverse))
print(f"Validation RMSE: {rmse:.4f}")

mape = mean_absolute_percentage_error(y_val_inverse, y_val_pred_inverse)
print(f"Validation MAPE: {mape:.4f}")